## Importing The Libraries

In [1]:
import os
import datetime
from utils import fetch_clean_data
from openai import OpenAI
from IPython.display import Markdown, display
from dotenv import load_dotenv
import json

## Configuration

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI()

In [3]:
def generate_markdown_summary(data: dict, today_date_str: str) -> str:
    """
    Send cleaned Bitcoin data to an LLM and receive a Markdown summary.
    """

    system_prompt = f"""
    You are a professional crypto analyst. Your job is to read the provided Bitcoin network data 
    and write a clear, structured report that can be read directly as a daily summary.

    Following are the rules that you must adhere to:
    - **IMPORTANT**: The summary title MUST use today's date: {today_date_str}. The title must be: "Bitcoin Daily Summary - {today_date_str}".
    - **CRITICAL**: Do NOT infer the reporting period from the data. The data contains historical records, but your report is for {today_date_str}.
    - Include **headings** for sections like "Market Overview", "Network Metrics Explained", "Miner Revenue Trends", and "Halving Outlook".
    - Use **bullet points** for key metrics.
    - Use a **table** for historical or time-series data if available.
    - Explain important terms (like hashrate, difficulty, transaction fees) in plain language.

    Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
    """

    # Convert the Python data dictionary into a clean JSON string for the prompt
    data_str = json.dumps(data, indent=2)

    user_prompt = f"""
    Today's date is {today_date_str}. Use this as the reference point for the report.

    The following data may contain historical records (e.g., from 2024), 
    but you must treat it as background context and write the summary as of {today_date_str}.

    Here is the data for you to summarize: 
    {data_str}
    """
    
    response = client.chat.completions.create(
        model= "gpt-4.1-mini", 
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    markdown_text = response.choices[0].message.content.strip()
    return markdown_text

## Main Function

In [ ]:
def main():
    # 0. Get today's date as a string
    today_str = datetime.datetime.now().strftime('%B %d, %Y')
    
    # 1. Fetch and clean data
    print("Fetching Bitcoin data...")
    data = fetch_clean_data()

    # 2. Generate Markdown summary
    print("Generating LLM summary...")
    markdown_report = generate_markdown_summary(data, today_str)

    # 3. Display Output
    display(Markdown(markdown_report))

if __name__ == "__main__":
    main()